# Dense Retrieval

## Setup

Load needed API keys and relevant Python libaries.

In [1]:
# !pip install cohere 
# !pip install weaviate-client Annoy

In [2]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [3]:
import cohere
co = cohere.Client("YOUR_API_KEY")

In [4]:
import weaviate
auth_config = weaviate.auth.AuthApiKey(api_key = "76320a90-53d8-42bc-b41d-678647c6672e")

In [6]:
client = weaviate.Client(url = "https://cohere-demo.weaviate.network/",
                         auth_client_secret = auth_config,
                         additional_headers = {"X-Cohere-Api-Key":
                                               "YOUR_API_KEY",
                                              })
client.is_ready()

True

## Part 1: Vector Database for semantic Search

In [7]:
def dense_retrieval(query, 
                    results_lang = "en",
                    properties = ["text", "title", "url", "views", "lang", "_additional {distance}"],
                    num_results = 5):

    nearText = {"concepts": [query]}
    
    # To filter by language
    where_filter = {"path": ["lang"],
                    "operator": "Equal",
                    "valueString": results_lang}
    response = (client.query
                .get("Articles", properties)
                .with_near_text(nearText)
                .with_where(where_filter)
                .with_limit(num_results)
                .do())

    result = response["data"]["Get"]["Articles"]

    return result

In [8]:
from utils import print_result

### Basic Query

In [9]:
query = "Who was James Clerk Maxwell?"
dense_retrieval_results = dense_retrieval(query)
print_result(dense_retrieval_results)

item 0
_additional:{'distance': -156.5397}

lang:en

text:James Clerk Maxwell (13 June 1831 – 5 November 1879) was a Scottish mathematician and scientist responsible for the classical theory of electromagnetic radiation, which was the first theory to describe electricity, magnetism and light as different manifestations of the same phenomenon. Maxwell's equations for electromagnetism have been called the "second great unification in physics" where the first one had been realised by Isaac Newton.

title:James Clerk Maxwell

url:https://en.wikipedia.org/wiki?curid=28989696

views:2000


item 1
_additional:{'distance': -156.27994}

lang:en

text:In October 1850, already an accomplished mathematician, Maxwell left Scotland for the University of Cambridge. He initially attended Peterhouse, but before the end of his first term transferred to Trinity, where he believed it would be easier to obtain a fellowship. At Trinity he was elected to the elite secret society known as the Cambridge Apostl

### Medium Query

In [10]:
query = "What is the Gamma function?"
dense_retrieval_results = dense_retrieval(query)
print_result(dense_retrieval_results)

item 0
_additional:{'distance': -151.89143}

lang:en

text:In mathematics, the gamma function (represented by , the capital letter gamma from the Greek alphabet) is one commonly used extension of the factorial function to complex numbers. The gamma function is defined for all complex numbers except the non-positive integers. For every positive integer , formula_1

title:Gamma function

url:https://en.wikipedia.org/wiki?curid=12316

views:2000


item 1
_additional:{'distance': -150.49522}

lang:en

text:In the context of technical and physical applications, e.g. with wave propagation, the functional equation

title:Gamma function

url:https://en.wikipedia.org/wiki?curid=12316

views:2000


item 2
_additional:{'distance': -150.30193}

lang:en

text:When seeking to approximate formula_20 for a complex number formula_21, it is effective to first compute formula_22 for some large integer formula_23. Use that to approximate a value for formula_24, and then use the recursion relation formula_

In [11]:
from utils import keyword_search

query = "What is the Gamma function?"
keyword_search_results = keyword_search(query, client)
print_result(keyword_search_results)

item 0
text:The name gamma function and the symbol were introduced by Adrien-Marie Legendre around 1811; Legendre also rewrote Euler's integral definition in its modern form. Although the symbol is an upper-case Greek gamma, there is no accepted standard for whether the function name should be written "gamma function" or "Gamma function" (some authors simply write "-function"). The alternative "pi function" notation due to Gauss is sometimes encountered in older literature, but Legendre's notation is dominant in modern works.

title:Gamma function

url:https://en.wikipedia.org/wiki?curid=12316


item 1
text:By taking limits, certain rational products with infinitely many factors can be evaluated in terms of the gamma function as well. Due to the Weierstrass factorization theorem, analytic functions can be written as infinite products, and these can sometimes be represented as finite products or quotients of the gamma function. We have already seen one striking example: the reflection f

### Complicated Query

In [12]:
from utils import keyword_search

query = "Tallest person in history?"
keyword_search_results = keyword_search(query, client)
print_result(keyword_search_results)

item 0
text:The population of Japan peaked at 128,083,960 in 2008. It had decreased by 2,373,960 by December 2020. In 2011, the economy of China became the world's second largest. Japan's economy descended to third largest by nominal GDP. Despite Japan's economic difficulties, this period also saw Japanese popular culture, including video games, anime, and manga, expanding worldwide, especially among young people. In March 2011, the Tokyo Skytree became the tallest tower in the world at , displacing the Canton Tower. It is the second tallest structure in the world after the Burj Khalifa ().

title:History of Japan

url:https://en.wikipedia.org/wiki?curid=25890428


item 1
text:Alpinism author Jon Krakauer (1997) wrote in "Into Thin Air" that it would be a bigger challenge to climb the second-highest peak of each continent, known as the Seven Second Summits – a feat that was not accomplished until January 2013. This discussion had previously been published in an article titled "The Seco

In [13]:
query = "Tallest person in history"
dense_retrieval_results = dense_retrieval(query)
print_result(dense_retrieval_results)

item 0
_additional:{'distance': -148.98888}

lang:en

text:Robert Pershing Wadlow (February 22, 1918 July 15, 1940), also known as the Alton Giant and the Giant of Illinois, was a man who was the tallest person in recorded history for whom there is irrefutable evidence. He was born and raised in Alton, Illinois, a small city near St. Louis, Missouri.

title:Robert Wadlow

url:https://en.wikipedia.org/wiki?curid=359117

views:3000


item 1
_additional:{'distance': -148.09317}

lang:en

text:Bol came from a family of extraordinarily tall men and women. He said: "My mother was , my father , and my sister is . And my great-grandfather was even taller—." His ethnic group, the Dinka, and the Nilotic people of which they are a part, are among the tallest populations in the world. Bol's hometown, Turalei, is the origin of other exceptionally tall people, including basketball player Ring Ayuel. "I was born in a village, where you cannot measure yourself," Bol reflected. "I learned I was 7 foot 

In [14]:
query = "أطول رجل في التاريخ"
dense_retrieval_results = dense_retrieval(query)
print_result(dense_retrieval_results)

item 0
_additional:{'distance': -147.44199}

lang:en

text:Robert Pershing Wadlow (February 22, 1918 July 15, 1940), also known as the Alton Giant and the Giant of Illinois, was a man who was the tallest person in recorded history for whom there is irrefutable evidence. He was born and raised in Alton, Illinois, a small city near St. Louis, Missouri.

title:Robert Wadlow

url:https://en.wikipedia.org/wiki?curid=359117

views:3000


item 1
_additional:{'distance': -147.09518}

lang:en

text:Kösen turned 40 years old on 10 December 2022. He celebrated his birthday a few days early by visiting the Ripley's Believe It or Not! museum in Orlando, Florida, USA and posing next to a life-sized statue of Robert Wadlow, the tallest man ever at 272 cm (8 ft 11.1 in).

title:Sultan Kösen

url:https://en.wikipedia.org/wiki?curid=8445237

views:2000


item 2
_additional:{'distance': -146.9144}

lang:en

text:Bol and Gheorghe Mureșan are the two tallest players in the history of the National Basketbal

In [15]:
query = "film about physics"
dense_retrieval_results = dense_retrieval(query)
print_result(dense_retrieval_results)

item 0
_additional:{'distance': -148.39377}

lang:en

text:The film was a co-production between the motion picture studios of Moving Picture Company, DNA Films, UK Film Council, and Ingenious Film Partners. Theatrically, it was commercially distributed by Fox Searchlight Pictures, while the 20th Century Fox Home Entertainment division released the film in the video rental market. "Sunshine" explores physics, science and religion. Following its wide release in theatres, the film garnered several award nominations for its acting, directing, and production merits. It also won an award for Best Technical Achievement for production designer Mark Tildesley from the British Independent Film Awards. was composed by John Murphy and was released by the Fox Music Group on 25 November 2008.

title:Sunshine (2007 film)

url:https://en.wikipedia.org/wiki?curid=3252860

views:2000


item 1
_additional:{'distance': -148.22672}

lang:en

text:Patrick O'Donnell, professor of physics at the University of

## Part 2: Building Semantic Search from Scratch

### Get the text archive:

In [16]:
from annoy import AnnoyIndex
import numpy as np
import pandas as pd
import re

In [17]:
text = """
Interstellar is a 2014 epic science fiction film co-written, directed, and produced by Christopher Nolan.
It stars Matthew McConaughey, Anne Hathaway, Jessica Chastain, Bill Irwin, Ellen Burstyn, Matt Damon, and Michael Caine.
Set in a dystopian future where humanity is struggling to survive, the film follows a group of astronauts who travel through a wormhole near Saturn in search of a new home for mankind.

Brothers Christopher and Jonathan Nolan wrote the screenplay, which had its origins in a script Jonathan developed in 2007.
Caltech theoretical physicist and 2017 Nobel laureate in Physics[4] Kip Thorne was an executive producer, acted as a scientific consultant, and wrote a tie-in book, The Science of Interstellar.
Cinematographer Hoyte van Hoytema shot it on 35 mm movie film in the Panavision anamorphic format and IMAX 70 mm.
Principal photography began in late 2013 and took place in Alberta, Iceland, and Los Angeles.
Interstellar uses extensive practical and miniature effects and the company Double Negative created additional digital effects.

Interstellar premiered on October 26, 2014, in Los Angeles.
In the United States, it was first released on film stock, expanding to venues using digital projectors.
The film had a worldwide gross over $677 million (and $773 million with subsequent re-releases), making it the tenth-highest grossing film of 2014.
It received acclaim for its performances, direction, screenplay, musical score, visual effects, ambition, themes, and emotional weight.
It has also received praise from many astronomers for its scientific accuracy and portrayal of theoretical astrophysics. Since its premiere, Interstellar gained a cult following,[5] and now is regarded by many sci-fi experts as one of the best science-fiction films of all time.
Interstellar was nominated for five awards at the 87th Academy Awards, winning Best Visual Effects, and received numerous other accolades"""

### Chunking: 

In [18]:
# Split into a list of sentences
texts = text.split('.')

# Clean up to remove empty spaces and new lines
texts = np.array([t.strip(' \n') for t in texts])

In [19]:
texts

array(['Interstellar is a 2014 epic science fiction film co-written, directed, and produced by Christopher Nolan',
       'It stars Matthew McConaughey, Anne Hathaway, Jessica Chastain, Bill Irwin, Ellen Burstyn, Matt Damon, and Michael Caine',
       'Set in a dystopian future where humanity is struggling to survive, the film follows a group of astronauts who travel through a wormhole near Saturn in search of a new home for mankind',
       'Brothers Christopher and Jonathan Nolan wrote the screenplay, which had its origins in a script Jonathan developed in 2007',
       'Caltech theoretical physicist and 2017 Nobel laureate in Physics[4] Kip Thorne was an executive producer, acted as a scientific consultant, and wrote a tie-in book, The Science of Interstellar',
       'Cinematographer Hoyte van Hoytema shot it on 35 mm movie film in the Panavision anamorphic format and IMAX 70 mm',
       'Principal photography began in late 2013 and took place in Alberta, Iceland, and Los Angeles',

In [20]:
# Split into a list of paragraphs
texts = text.split('\n\n')

# Clean up to remove empty spaces and new lines
texts = np.array([t.strip(' \n') for t in texts])

In [21]:
texts

array(['Interstellar is a 2014 epic science fiction film co-written, directed, and produced by Christopher Nolan.\nIt stars Matthew McConaughey, Anne Hathaway, Jessica Chastain, Bill Irwin, Ellen Burstyn, Matt Damon, and Michael Caine.\nSet in a dystopian future where humanity is struggling to survive, the film follows a group of astronauts who travel through a wormhole near Saturn in search of a new home for mankind.',
       'Brothers Christopher and Jonathan Nolan wrote the screenplay, which had its origins in a script Jonathan developed in 2007.\nCaltech theoretical physicist and 2017 Nobel laureate in Physics[4] Kip Thorne was an executive producer, acted as a scientific consultant, and wrote a tie-in book, The Science of Interstellar.\nCinematographer Hoyte van Hoytema shot it on 35 mm movie film in the Panavision anamorphic format and IMAX 70 mm.\nPrincipal photography began in late 2013 and took place in Alberta, Iceland, and Los Angeles.\nInterstellar uses extensive practical 

In [22]:
# Split into a list of sentences
texts = text.split('.')

# Clean up to remove empty spaces and new lines
texts = np.array([t.strip(' \n') for t in texts])

In [23]:
title = 'Interstellar (film)'

texts = np.array([f"{title} {t}" for t in texts])

In [24]:
texts

array(['Interstellar (film) Interstellar is a 2014 epic science fiction film co-written, directed, and produced by Christopher Nolan',
       'Interstellar (film) It stars Matthew McConaughey, Anne Hathaway, Jessica Chastain, Bill Irwin, Ellen Burstyn, Matt Damon, and Michael Caine',
       'Interstellar (film) Set in a dystopian future where humanity is struggling to survive, the film follows a group of astronauts who travel through a wormhole near Saturn in search of a new home for mankind',
       'Interstellar (film) Brothers Christopher and Jonathan Nolan wrote the screenplay, which had its origins in a script Jonathan developed in 2007',
       'Interstellar (film) Caltech theoretical physicist and 2017 Nobel laureate in Physics[4] Kip Thorne was an executive producer, acted as a scientific consultant, and wrote a tie-in book, The Science of Interstellar',
       'Interstellar (film) Cinematographer Hoyte van Hoytema shot it on 35 mm movie film in the Panavision anamorphic format

### Get the embeddings:

In [26]:
response = co.embed(texts = texts.tolist()).embeddings

In [27]:
embeds = np.array(response)
embeds.shape

(15, 4096)

### Create the search index:

In [28]:
search_index = AnnoyIndex(embeds.shape[1], "angular")
# Add all the vectors to the search index
for i in range(len(embeds)):
    search_index.add_item(i, embeds[i])

search_index.build(10) # 10 trees
search_index.save("test.ann")

True

In [29]:
pd.set_option("display.max_colwidth", None)

def search(query):

  # Get the query's embedding
  query_embed = co.embed(texts=[query]).embeddings

  # Retrieve the nearest neighbors
  similar_item_ids = search_index.get_nns_by_vector(query_embed[0],
                                                    3,
                                                  include_distances=True)
  # Format the results
  results = pd.DataFrame(data = {"texts": texts[similar_item_ids[0]],
                                 "distance": similar_item_ids[1]})

  print(texts[similar_item_ids[0]])
    
  return results

In [30]:
query = "How much did the film make?"
search(query)

['Interstellar (film) The film had a worldwide gross over $677 million (and $773 million with subsequent re-releases), making it the tenth-highest grossing film of 2014'
 'Interstellar (film) Interstellar premiered on October 26, 2014, in Los Angeles'
 'Interstellar (film) In the United States, it was first released on film stock, expanding to venues using digital projectors']


,texts,distance
0,"Interstellar (film) The film had a worldwide gross over $677 million (and $773 million with subsequent re-releases), making it the tenth-highest grossing film of 2014",1.019056
1,"Interstellar (film) Interstellar premiered on October 26, 2014, in Los Angeles",1.144951
2,"Interstellar (film) In the United States, it was first released on film stock, expanding to venues using digital projectors",1.167268
